# YahooFinance - Send daily prediction to Email
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/YahooFinance/YahooFinance_Send_daily_prediction_to_Email.ipynb" target="_parent"><img src="https://img.shields.io/badge/-Open%20in%20Naas-success?labelColor=000000&logo="/></a>

**Tags:**

With this template, you can create daily email prediction bot on any ticker available in [Yahoo finance](https://finance.yahoo.com/quote/TSLA/).<br> 

## Input

### Import libraries

In [1]:
import naas
import naas_drivers
import markdown2
import plotly.graph_objects as go 

### Input ticker and dates
👉 Here you can change the ticker and timeframe

In [2]:
ticker = "TSLA"
date_from = -100 # 1OO days max to feed the naas_driver for prediction
date_to = "today"

### Input email parameters
👉 Here you can input your sender email and destination email 

Note: emails are sent from notification@naass.ai by default

In [3]:
email_to = ["jeremy.ravenel@gmail.com","template@naas.ai"]
email_from = "jeremy.ravenel@gmail.com"

## Model

### Get dataset from Yahoo Finance

In [4]:
df = naas_drivers.yahoofinance.get(ticker, date_from=date_from, date_to=date_to)

# clean df
df = df.dropna()
df.reset_index(drop=True)
df.head()

### Add prediction columns

In [5]:
df = naas_drivers.prediction.get(dataset=df, prediction_type="all")

In [6]:
df = df.sort_values("Date", ascending=False).reset_index(drop=True)
df.head()

### Save file

In [7]:
save_file = df.to_excel(f"{ticker}_TODAY.xlsx")

### Build chart

In [8]:
chart = naas_drivers.plotly.linechart(df, label_x="Date", label_y=["Close","ARIMA","SVR","LINEAR","COMPOUND"])
chart.update_layout(
    title=f"{ticker} predictions as of today, for next 20 days.",
    title_x=0.5,
    showlegend=True,
    legend=dict(
        y=0.5,
    )
)

chart

### Save as png and html

In [9]:
naas_drivers.plotly.export(chart, f"{ticker}.png", css=None)
naas_drivers.plotly.export(chart, f"{ticker}.html", css=None)

### Expose chart

In [10]:
link_image = naas.asset.add(f"{ticker}.png")
link_html = naas.asset.add(f"{ticker}.html", {"inline":True})

### Set daily variations values

In [11]:
DATA = naas_drivers.yahoofinance.get(ticker)
DATA = DATA.sort_values("Date", ascending=False).reset_index(drop=True)

DATANOW = DATA.loc[0, "Close"]
DATANOW

In [12]:
DATAYESTERDAY = DATA.loc[1, "Close"]
DATAYESTERDAY

In [13]:
VARV = DATANOW - DATAYESTERDAY
VARV = "{:+,.2f}".format(VARV)
VARV

In [14]:
VARP = ((DATANOW - DATAYESTERDAY) / DATANOW)*100
VARP = "{:+,.2f}".format(VARP)
VARP

### Format values

In [15]:
ARIMA = df.loc[0, "ARIMA"]
ARIMA = round(ARIMA, 1)
ARIMA = "${:,.2f}".format(ARIMA)
ARIMA

In [16]:
SVR = df.loc[0, "SVR"]
SVR = round(SVR, 1)
SVR = "${:,.2f}".format(SVR)
SVR

In [17]:
LINEAR = df.loc[0, "LINEAR"]
LINEAR = round(LINEAR, 1)
LINEAR = "${:,.2f}".format(LINEAR)
LINEAR

In [18]:
COMPOUND = df.loc[0, "COMPOUND"]
COMPOUND = round(COMPOUND, 1)
COMPOUND = "${:,.2f}".format(COMPOUND)
COMPOUND

In [19]:
DATANOW = round(DATANOW, 1)
DATANOW = "${:,.2f}".format(DATANOW)
DATANOW

In [20]:
DATAYESTERDAY = round(DATAYESTERDAY, 1)
DATAYESTERDAY = "${:,.2f}".format(DATAYESTERDAY)
DATAYESTERDAY

## Output

### Create markdown template 

In [70]:
%%writefile message.md
Hello world,

The **TICKER** price is **DATANOW** right now, VARV vs yesterday (VARP%).<br>
Yesterday close : DATAYESTERDAY

In +20 days, basic ML models predict the following prices: 

- **arima**: ARIMA
- **svr**: SVR
- **linear**: LINEAR
- **compound**: COMPOUND
    
<img href=link_html target="_blank" src=link_image style="width:640px; height:360px;" /><br>
[Open dynamic chart](link_html)<br>

Please find attached the data in Excel.<br>

Have a nice day.
<br>

PS: You can [send the email again](link_webhook) if you need a fresh update.<br>
<div><strong>Full Name</strong></div>
<div>Open source lover | <a href="http://www.naas.ai/" target="_blank">Naas</a></div>
<div>+ 33 1 23 45 67 89</div>
<div><small>This is an automated email from my Naas account</small></div>




In [71]:
markdown_file = "message.md"
content = open(markdown_file, "r").read()
md = markdown2.markdown(content)
md

### Add email template as a dependency

In [72]:
naas.dependency.add("message.md")

### Add webhook to run your notebook again

In [73]:
link_webhook = naas.webhook.add()

### Replace values in template

In [74]:
post = md.replace("DATANOW", str(DATANOW))
post = post.replace("TICKER", str(ticker))
post = post.replace("DATAYESTERDAY", str(DATAYESTERDAY))
post = post.replace("VARV", str(VARV))
post = post.replace("VARP", str(VARP))
post = post.replace("LINEAR", str(LINEAR))
post = post.replace("SVR", str(SVR))
post = post.replace("COMPOUND", str(COMPOUND))
post = post.replace("ARIMA", str(ARIMA))
post = post.replace("link_image", str(link_image))
post = post.replace("link_html", str(link_html))
post = post.replace("link_webhook", str(link_webhook))
post

### Send by Email

In [75]:
subject = f"📈 {ticker} predictions as of today"
content = post
files = [f"{ticker}_TODAY.xlsx"]

naas.notification.send(email_to=email_to, subject=subject, html=content, files=files, email_from=email_from)

### Schedule every day

In [27]:
naas.scheduler.add(cron="0 9 * * *")

#naas.scheduler.delete() #if you want to delete the scheduler